In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [21]:
chromedriver = '/Workspace/02. DataAnalysis/05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [23]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url + quote('가을장마')

In [24]:
driver.get(url)

In [26]:
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(40):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [27]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [28]:
lis = soup.select('li.bx._svp_item')
len(lis)

153

In [29]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, date, content

('가을장마가 시작되네요...',
 '6일 전',
 'ㅠ 저번달에는 장마로 인해 농부삼촌네 절임공장이 #산사태로 인해 #수해피해를 보면서 이번 #태풍 #오마이스 가 지나고 계속된 #가을장마 소식에 머리가 지끈지끈 합니다.... 올해도 8월 7일부터 1차 배추모종을 담는것부터...')

In [30]:
title_list, content_list, date_list = [], [], []
for i, li in enumerate(lis):
    try:
        adv = li.select_one('a.link_ad').get_text()
        if adv.strip() == '광고':
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

In [31]:
df = pd.DataFrame({
    'title': title_list, 'date' : date_list, 'content' : content_list
})
df.head(5)

,title,date,content
0,가을장마가 시작되네요...,6일 전,ㅠ 저번달에는 장마로 인해 농부삼촌네 절임공장이 #산사태로 인해 #수해피해를 보면서...
1,가을 장마에 하기 좋은 과학실험키트 5,어제,가을 장마를 이겨낼 수 있는 #과학실험키트 5가지를 소개합니다. 습기 NO! 매직 ...
2,가을장마(feat. 김장 배추 키우기),6일 전,뭔 가을장마가 이리 긴 것인가... 그렇게 여름에 비 좀 오라고~오라고 할 때는 다...
3,가을장마 시작 - 다육이들 비닐 덮개 씌우기,2021.08.21.,말복 지나고 나니 좀 선선해져서 좋다고 했는데 가을 장마로 가을 수확을 위한 농작물...
4,가을 장마,2021.08.22.,뒤늦은 장마는 경남 남해안은 특히 더 비를 쏟아부었으니 과연 우리 집 잔디밭에는 웅...


In [32]:
df.to_csv('data/rain_blog.txt')

In [33]:
driver.close()